In [7]:
def vacuum_cleaner():
    loc = input("Vacuum location: ").upper()
    
    room = {}
    for r in ['A', 'B', 'C']:
        room[r] = int(input(r + " status: "))
    
    goal = {'A': 0, 'B': 0, 'C': 0}
    cost = 0
    
    print("state", room)
    print("vacuum in", loc)
    
    order = ['A', 'B', 'C']
    start = order.index(loc)
    
    if room[loc] == 1:
        print(loc, "dirty")
        print("cleaning", loc)
        room[loc] = 0
        cost += 1
    print(loc, "clean")
    
    left = start > 0 and (room['A'] == 1 or room['C'] == 1)
    
    if left:
        for i in range(start, -1, -1):
            r = order[i]
            if room[r] == 1:
                print(r, "dirty")
                print("cleaning", r)
                room[r] = 0
                cost += 1
            print(r, "clean")
            
            if i > 0:
                print("move to", order[i-1])
                cost += 1
        
        if room['C'] == 1:
            print("move to C")
            cost += 1
            print("cleaning C")
            room['C'] = 0
            cost += 1
    else:
        for i in range(start, len(order)):
            r = order[i]
            if room[r] == 1:
                print(r, "dirty")
                print("cleaning", r)
                room[r] = 0
                cost += 1
            print(r, "clean")
            
            if i < len(order) - 1:
                print("move to", order[i+1])
                cost += 1
        
        if room['A'] == 1:
            print("move to A")
            cost += 1
            print("cleaning A")
            room['A'] = 0
            cost += 1
    
    print("goal", room)
    print("cost", cost)
    
vacuum_cleaner()


state {'A': 1, 'B': 1, 'C': 1}
vacuum in A
A dirty
cleaning A
A clean
A clean
move to B
B dirty
cleaning B
B clean
move to C
C dirty
cleaning C
C clean
goal {'A': 0, 'B': 0, 'C': 0}
cost 5


In [7]:
import random

grid_size = 5

def create_grid():
    grid = [['.' for _ in range(grid_size)] for _ in range(grid_size)]
    goal_x, goal_y = random.randint(0, 4), random.randint(0, 4)
    grid[goal_x][goal_y] = 'G'
    for _ in range(5):  # Randomly place 5 obstacles
        x, y = random.randint(0, 4), random.randint(0, 4)
        if grid[x][y] == '.':
            grid[x][y] = 'O'
    return grid, goal_x, goal_y

def print_grid(grid, agent_x, agent_y):
    for i in range(grid_size):
        for j in range(grid_size):
            if i == agent_x and j == agent_y:
                print('A', end=' ')
            else:
                print(grid[i][j], end=' ')
        print()
    print('-' * 10)

class ModelBasedAgent:
    def __init__(self, grid, goal_x, goal_y):
        self.grid = grid
        self.goal_x, self.goal_y = goal_x, goal_y
        self.agent_x, self.agent_y = 0, 0
        self.known_grid = [['?' for _ in range(grid_size)] for _ in range(grid_size)]
        self.known_grid[self.agent_x][self.agent_y] = 'A'
    
    def perceive(self):
        surroundings = {}
        for dx, dy, direction in [(-1, 0, 'Up'), (1, 0, 'Down'), (0, -1, 'Left'), (0, 1, 'Right')]:
            nx, ny = self.agent_x + dx, self.agent_y + dy
            if 0 <= nx < grid_size and 0 <= ny < grid_size:
                surroundings[direction] = self.grid[nx][ny]
                self.known_grid[nx][ny] = self.grid[nx][ny]
        return surroundings
    
    def decide_move(self, surroundings):
        if (self.agent_x, self.agent_y) == (self.goal_x, self.goal_y):
            return None
        for direction, content in surroundings.items():
            if content == 'G':
                return direction
        possible_moves = [d for d, c in surroundings.items() if c == '.']
        return random.choice(possible_moves) if possible_moves else None
    
    def move(self, direction):
        if direction == 'Up':
            self.agent_x -= 1
        elif direction == 'Down':
            self.agent_x += 1
        elif direction == 'Left':
            self.agent_y -= 1
        elif direction == 'Right':
            self.agent_y += 1
    
    def update_model(self):
        self.known_grid[self.agent_x][self.agent_y] = 'A'
    
    def run(self, steps=10):
        for _ in range(steps):
            print_grid(self.grid, self.agent_x, self.agent_y)
            surroundings = self.perceive()
            move_direction = self.decide_move(surroundings)
            if move_direction is None:
                print("Agent reached goal or is stuck.")
                break
            self.move(move_direction)
            self.update_model()
        print("Final Grid State:")
        print_grid(self.grid, self.agent_x, self.agent_y)

# Run the simulation
grid, goal_x, goal_y = create_grid()
agent = ModelBasedAgent(grid, goal_x, goal_y)
agent.run()


A . O O . 
. . . . . 
. . . . O 
. . . . O 
. G . . . 
----------
. A O O . 
. . . . . 
. . . . O 
. . . . O 
. G . . . 
----------
. . O O . 
. A . . . 
. . . . O 
. . . . O 
. G . . . 
----------
. . O O . 
. . . . . 
. A . . O 
. . . . O 
. G . . . 
----------
. . O O . 
. . . . . 
A . . . O 
. . . . O 
. G . . . 
----------
. . O O . 
A . . . . 
. . . . O 
. . . . O 
. G . . . 
----------
. . O O . 
. . . . . 
A . . . O 
. . . . O 
. G . . . 
----------
. . O O . 
. . . . . 
. A . . O 
. . . . O 
. G . . . 
----------
. . O O . 
. . . . . 
. . . . O 
. A . . O 
. G . . . 
----------
. . O O . 
. . . . . 
. . . . O 
. . . . O 
. A . . . 
----------
Agent reached goal or is stuck.
Final Grid State:
. . O O . 
. . . . . 
. . . . O 
. . . . O 
. A . . . 
----------


In [8]:
import random
import copy

grid_size = 5

def create_grid():
    grid = [['.' for _ in range(grid_size)] for _ in range(grid_size)]
    goal_x, goal_y = random.randint(0, 4), random.randint(0, 4)
    grid[goal_x][goal_y] = 'G'
    for _ in range(5):  # Randomly place 5 obstacles
        x, y = random.randint(0, 4), random.randint(0, 4)
        if grid[x][y] == '.':
            grid[x][y] = 'O'
    start_x, start_y = random.randint(0, 4), random.randint(0, 4)
    while (start_x, start_y) == (goal_x, goal_y) or grid[start_x][start_y] == 'O':
        start_x, start_y = random.randint(0, 4), random.randint(0, 4)
    return grid, goal_x, goal_y, start_x, start_y

def print_grid(grid, agent_x, agent_y):
    for i in range(grid_size):
        for j in range(grid_size):
            if i == agent_x and j == agent_y:
                print('A', end=' ')
            else:
                print(grid[i][j], end=' ')
        print()
    print('-' * 10)

class SimpleReflexAgent:
    def __init__(self, grid, goal_x, goal_y, start_x, start_y):
        self.grid = grid
        self.goal_x, self.goal_y = goal_x, goal_y
        self.agent_x, self.agent_y = start_x, start_y
        self.steps = 0

    def perceive(self):
        return self.grid[self.agent_x][self.agent_y]

    def decide_move(self):
        if (self.agent_x, self.agent_y) == (self.goal_x, self.goal_y):
            return None
        moves = [(0, 1, 'Right'), (0, -1, 'Left'), (1, 0, 'Down'), (-1, 0, 'Up')]
        random.shuffle(moves)
        for dx, dy, direction in moves:
            nx, ny = self.agent_x + dx, self.agent_y + dy
            if 0 <= nx < grid_size and 0 <= ny < grid_size and self.grid[nx][ny] != 'O':
                return direction
        return None

    def move(self, direction):
        if direction == 'Up':
            self.agent_x -= 1
        elif direction == 'Down':
            self.agent_x += 1
        elif direction == 'Left':
            self.agent_y -= 1
        elif direction == 'Right':
            self.agent_y += 1
        self.steps += 1

    def run(self):
        while self.steps < 20:
            print_grid(self.grid, self.agent_x, self.agent_y)
            if self.perceive() == 'G':
                print("Simple Reflex Agent reached the goal!")
                break
            move_direction = self.decide_move()
            if move_direction is None:
                print("Simple Reflex Agent is stuck.")
                break
            self.move(move_direction)
        print(f"Steps Taken: {self.steps}\n")

grid, goal_x, goal_y, start_x, start_y = create_grid()
print("Environment:")
print_grid(grid, -1, -1)
print("Running Simple Reflex Agent:")
simple_agent = SimpleReflexAgent(copy.deepcopy(grid), goal_x, goal_y, start_x, start_y)
simple_agent.run()

Environment:
. . G . . 
. . . . . 
. . O . . 
. . . . . 
O O . O . 
----------
Running Simple Reflex Agent:
. . G . . 
. . . . . 
. . O A . 
. . . . . 
O O . O . 
----------
. . G . . 
. . . A . 
. . O . . 
. . . . . 
O O . O . 
----------
. . G . . 
. . . . . 
. . O A . 
. . . . . 
O O . O . 
----------
. . G . . 
. . . A . 
. . O . . 
. . . . . 
O O . O . 
----------
. . G A . 
. . . . . 
. . O . . 
. . . . . 
O O . O . 
----------
. . A . . 
. . . . . 
. . O . . 
. . . . . 
O O . O . 
----------
Simple Reflex Agent reached the goal!
Steps Taken: 5



In [ ]:
import random
import copy

grid_size = 5

def create_grid(test_case):
    """Creates different grid configurations based on the test case."""
    grid = [['.' for _ in range(grid_size)] for _ in range(grid_size)]
    
    if test_case == 1: 
        goal_x, goal_y = 1, 1
        obstacles = [(2, 2)]
    
    elif test_case == 2: 
        goal_x, goal_y = 4, 4
        obstacles = [(1, 1), (1, 3), (2, 2), (3, 1), (3, 3)]
    
    elif test_case == 3:  
        goal_x, goal_y = 4, 0
        obstacles = [(1, 2), (2, 1), (3, 3), (4, 2)]
    
    elif test_case == 4:  
        goal_x, goal_y = 3, 3
        obstacles = [(1, 1), (1, 2), (2, 1), (3, 1), (3, 2)]
    
    elif test_case == 5:
        goal_x, goal_y = 4, 4
        obstacles = []

    grid[goal_x][goal_y] = 'G'
    
    for x, y in obstacles:
        grid[x][y] = 'O'

    start_x, start_y = 0, 0
    
    return grid, goal_x, goal_y, start_x, start_y

def print_grid(grid, agent_x, agent_y):
    for i in range(grid_size):
        for j in range(grid_size):
            if i == agent_x and j == agent_y:
                print('A', end=' ')
            else:
                print(grid[i][j], end=' ')
        print()
    print('-' * 10)

class ModelBasedAgent:
    def __init__(self, grid, goal_x, goal_y, start_x, start_y):
        self.grid = grid
        self.goal_x, self.goal_y = goal_x, goal_y
        self.agent_x, self.agent_y = start_x, start_y
        self.steps = 0

    def perceive(self):
        surroundings = {}
        for dx, dy, direction in [(-1, 0, 'Up'), (1, 0, 'Down'), (0, -1, 'Left'), (0, 1, 'Right')]:
            nx, ny = self.agent_x + dx, self.agent_y + dy
            if 0 <= nx < grid_size and 0 <= ny < grid_size:
                surroundings[direction] = self.grid[nx][ny]
        return surroundings

    def decide_move(self, surroundings):
        if (self.agent_x, self.agent_y) == (self.goal_x, self.goal_y):
            return None
        for direction, content in surroundings.items():
            if content == 'G':
                return direction
        possible_moves = [d for d, c in surroundings.items() if c == '.']
        return random.choice(possible_moves) if possible_moves else None

    def move(self, direction):
        if direction == 'Up':
            self.agent_x -= 1
        elif direction == 'Down':
            self.agent_x += 1
        elif direction == 'Left':
            self.agent_y -= 1
        elif direction == 'Right':
            self.agent_y += 1
        self.steps += 1

    def run(self, max_steps=20):
        while self.steps < max_steps:
            if (self.agent_x, self.agent_y) == (self.goal_x, self.goal_y):
                return self.steps
            surroundings = self.perceive()
            move_direction = self.decide_move(surroundings)
            if move_direction is None:
                return self.steps
            self.move(move_direction)
        return self.steps

class SimpleReflexAgent:
    def __init__(self, grid, goal_x, goal_y, start_x, start_y):
        self.grid = grid
        self.goal_x, self.goal_y = goal_x, goal_y
        self.agent_x, self.agent_y = start_x, start_y
        self.steps = 0

    def decide_move(self):
        if (self.agent_x, self.agent_y) == (self.goal_x, self.goal_y):
            return None
        moves = [(0, 1, 'Right'), (0, -1, 'Left'), (1, 0, 'Down'), (-1, 0, 'Up')]
        random.shuffle(moves)
        for dx, dy, direction in moves:
            nx, ny = self.agent_x + dx, self.agent_y + dy
            if 0 <= nx < grid_size and 0 <= ny < grid_size and self.grid[nx][ny] != 'O':
                return direction
        return None

    def move(self, direction):
        if direction == 'Up':
            self.agent_x -= 1
        elif direction == 'Down':
            self.agent_x += 1
        elif direction == 'Left':
            self.agent_y -= 1
        elif direction == 'Right':
            self.agent_y += 1
        self.steps += 1

    def run(self, max_steps=20):
        while self.steps < max_steps:
            if (self.agent_x, self.agent_y) == (self.goal_x, self.goal_y):
                return self.steps
            move_direction = self.decide_move()
            if move_direction is None:
                return self.steps
            self.move(move_direction)
        return self.steps

def run_test_cases():
    for i in range(1, 6):
        print(f"\nTest Case {i}:")
        grid, goal_x, goal_y, start_x, start_y = create_grid(i)
        print_grid(grid, -1, -1)

        model_agent = ModelBasedAgent(copy.deepcopy(grid), goal_x, goal_y, start_x, start_y)
        reflex_agent = SimpleReflexAgent(copy.deepcopy(grid), goal_x, goal_y, start_x, start_y)

        model_steps = model_agent.run()
        reflex_steps = reflex_agent.run()

        print(f"Model-Based Agent Steps: {model_steps}")
        print(f"Simple Reflex Agent Steps: {reflex_steps}")

        print("Comparison:")
        print(f" - Efficiency: Model-Based ({model_steps} steps) vs. Reflex ({reflex_steps} steps)")
        print(f" - Adaptability: Reflex agent struggles more in blocked scenarios.")
        print(f" - Decision Quality: Model agent makes informed moves, Reflex is more random.")
        print("=" * 30)

run_test_cases()



Test Case 1:
. . . . . 
. G . . . 
. . O . . 
. . . . . 
. . . . . 
----------
Model-Based Agent Steps: 2
Simple Reflex Agent Steps: 20
Comparison:
 - Efficiency: Model-Based (2 steps) vs. Reflex (20 steps)
 - Adaptability: Reflex agent struggles more in blocked scenarios.
 - Decision Quality: Model agent makes informed moves, Reflex is more random.

Test Case 2:
. . . . . 
. O . O . 
. . O . . 
. O . O . 
. . . . G 
----------
Model-Based Agent Steps: 20
Simple Reflex Agent Steps: 20
Comparison:
 - Efficiency: Model-Based (20 steps) vs. Reflex (20 steps)
 - Adaptability: Reflex agent struggles more in blocked scenarios.
 - Decision Quality: Model agent makes informed moves, Reflex is more random.

Test Case 3:
. . . . . 
. . O . . 
. O . . . 
. . . O . 
G . O . . 
----------
Model-Based Agent Steps: 20
Simple Reflex Agent Steps: 20
Comparison:
 - Efficiency: Model-Based (20 steps) vs. Reflex (20 steps)
 - Adaptability: Reflex agent struggles more in blocked scenarios.
 - Decision Qua